# Notebook for adding top_sites

These come in batches from Matthias. Last time, all the data were from the same date.

Assuming that continues, just set the DATE below to that date PLUS one day.

In [1]:
import pandas as pd

MEASUREMENTS_TIME = pd.Timestamp('2021-01-02')

In [2]:
from os.path import join
from os import listdir
from src.shared.utils import get_country
from datetime import datetime

## Parsing the dataframes

In [3]:
def parse_df (df: pd.DataFrame) -> pd.DataFrame:
    '''
    Returns a df with columns
    name, marketshare
    '''
    def percentify (x):
        try:
            n = x.split('%')[0]
            return float(n)/100
        except:
            return 0
    # name of columns where percentages are
    perc_col_name = [c for c in df.columns if c.startswith('Percentage')][0]
    df['marketshare'] = df[perc_col_name].apply(percentify)
    # if this is a heirarchical csv,
    # get top-level entries only
    if 'Rank' in df.columns:
        df['top-level'] = df['Rank'].apply(lambda x: str(x).endswith('.0'))
        df = df[df['top-level']==True]
        # get names from 1st column
        n = df.columns[1]
    else:
        # get names from 0th column
        n = df.columns[0]
    # get jurisdictions
    df['name'] = df[n]
    # remove 'and territories' for server locations
    df['name'] = df['name'].apply(lambda x: x.split(' and territories')[0])
    df['jurisdiction_alpha2'] = df['name'].apply(get_country)
    return df[['name', 'marketshare', 'jurisdiction_alpha2']]

In [4]:
ex_fn = listdir('top-sites')[1]
ex_df = pd.read_csv(join('top-sites', ex_fn))
parse_df(ex_df)

,name,marketshare,jurisdiction_alpha2
0,United States,0.4323,US
1,Germany,0.1899,DE
2,France,0.1329,FR
3,China,0.1147,CN
4,Japan,0.0576,JP
...,...,...,...
78,Armenia,0.0002,AM
79,Panama,0.0002,PA
80,Kuwait,0.0002,KW
81,Cambodia,0.0002,KH


## Extracting market/top-n from filenames

In [5]:
dfs = []
for my_dir in listdir('top-sites'):
    fn = my_dir.split('.csv')[0]
    if fn.split('-')[1]=='hierarchy':
        market, h, top_n, date_str  = fn.split('-')
        date = datetime.strptime(date_str, '%Y%M')
        print(market, top_n, date)
        df = pd.read_csv(join('top-sites', my_dir))
        df = parse_df(df)
        df['measurement_scope'] = top_n
        df['market'] = market
        df['date'] = date
        dfs.append(df)

dns_servers top_10k 2021-01-01 00:06:00
server_locations top_10k 2021-01-01 00:06:00
ssl_certificate top_10k 2021-01-01 00:06:00
data_center top_1k 2021-01-01 00:06:00
dns_servers top_1k 2021-01-01 00:06:00
web_hosting top_10k 2021-01-01 00:06:00
server_locations top_1k 2021-01-01 00:06:00
web_hosting top_1k 2021-01-01 00:06:00
ssl_certificate top_1k 2021-01-01 00:06:00
data_center top_10k 2021-01-01 00:06:00


/tmp/ipykernel_182070/103856182.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['name'] = df[n]
/tmp/ipykernel_182070/103856182.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['name'] = df['name'].apply(lambda x: x.split(' and territories')[0])
/tmp/ipykernel_182070/103856182.py:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org

In [6]:
pd.concat(dfs).to_csv('out/top-sites-combined.csv')

# Simple analyseshh,,  hh,,  

In [7]:
df = pd.read_csv('out/top-sites-combined.csv').drop('Unnamed: 0', axis=1)

In [9]:
df

,name,marketshare,jurisdiction_alpha2,measurement_scope,market,date
0,Cloudflare,0.2573,US,top_10k,dns_servers,2021-01-01 00:06:00
1,Amazon,0.2405,US,top_10k,dns_servers,2021-01-01 00:06:00
2,Akamai,0.0548,US,top_10k,dns_servers,2021-01-01 00:06:00
3,Google,0.0283,US,top_10k,dns_servers,2021-01-01 00:06:00
4,NS1,0.0275,US,top_10k,dns_servers,2021-01-01 00:06:00
...,...,...,...,...,...,...
920,Sedmi Odjel,0.0001,NaN,top_10k,data_center,2021-01-01 00:06:00
921,Alwaysdata,0.0001,NaN,top_10k,data_center,2021-01-01 00:06:00
922,Kamatera,0.0001,NaN,top_10k,data_center,2021-01-01 00:06:00
923,Telecom Algeria,0.0001,NaN,top_10k,data_center,2021-01-01 00:06:00


In [10]:
df.market = df.market.replace({
    'dns_servers': 'dns-server',
    'serer_locations': 'server-location',
    'data_center': 'data-centers',
    'ssl_certificate': 'ssl-certificate',
})

# TODO: Write to database

NOTE FOR README: In 'heirarchical' files, i took the top-level only (e.g., 'DigiCert Group' vs. 'DigiCert' + its other subsidiaries).


In [11]:
from config import config

In [12]:
postgres_config = config['postgres']


In [13]:
import psycopg2
conn = psycopg2.connect(**postgres_config)
cur = conn.cursor()

In [14]:
from imp import reload

In [15]:
# conn.commit()

In [16]:
# provider marketshare for each
import src.w3techs.types
reload(src.w3techs.types)
from src.w3techs.types import ProviderMarketshare
from src.shared.types import Alpha2


for i, row in df.iterrows():
    try:
        alpha2 = Alpha2(row.jurisdiction_alpha2)
    except:
        alpha2 = None
    marketshare = ProviderMarketshare(
        row['name'],
        None,
        alpha2,
        row.measurement_scope,
        row.market, 
        float(row['marketshare']),
        pd.Timestamp(row.date))
    marketshare.write_to_db(cur, conn, commit=False)

In [17]:
conn.commit()

## find pop weighted gini for top 1k and top 10k

In [19]:
from src.w3techs.collect import included_markets 
import src.w3techs.utils as utils

# time = pd.Timestamp(df.date.unique()[0], tz='America/Los_Angeles')
for measurement_scope in ['top_1k', 'top_10k']:
    print(measurement_scope)
    for market in included_markets:
        pop_weighted_gini = utils.population_weighted_gini(
             cur, measurement_scope, market,  MEASUREMENTS_TIME)
        if pop_weighted_gini:
            print(f'[X] {market}')
            pop_weighted_gini.write_to_db(cur, conn)
        else:
            print(f'[ ] {market}')

top_1k
[ ] web-hosting
[X] ssl-certificate
[ ] proxy
[X] data-centers
[X] dns-server
[ ] server-location
[ ] top-level-domain
top_10k
[ ] web-hosting
[X] ssl-certificate
[ ] proxy
[X] data-centers
[X] dns-server
[ ] server-location
[ ] top-level-domain
